<h1>Импорты</h1>

In [7]:
import pandas as pd
import numpy as np
import pyodbc
import random
import math
from sklearn.preprocessing import OrdinalEncoder
from datetime import datetime

In [8]:
# Connect to old database
db_conn = pyodbc.connect('Driver={SQL Server};'
#                       'Server=BAMBI;'
                         'Server=LAPTOP-JSR6TV0G;'
                      'Database=Pro_t;'
                      'Trusted_Connection=yes;')

In [640]:
# Connect to archive
archive_conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=progress;'
                      'Database=ArchDocT;'
                      'Trusted_Connection=yes;')

In [50]:
# Connect to new database
try:
    conn = pyodbc.connect('Driver={SQL Server};'
#                       'Server=BAMBI;'
                          'Server=LAPTOP-JSR6TV0G;'
                      'Database=DocumentsKM;'
                      'Trusted_Connection=yes;')
    cursor = conn.cursor()
except Exception as err:
    cursor = None
    print("Connection error:", err)
    
# Check if the connection was valid
if cursor != None:
    print("Connection successful")

Connection successful


In [10]:
def to_pascal_case(snake_str):
    components = snake_str.split('_')
    return components[0].title() + ''.join(x.title() for x in components[1:])

In [11]:
def execute_values(df, table_snake_case, has_dentity=True):
    df.columns = map(to_pascal_case, df.columns)
    df = df.replace({False: 0, True: 1})
    table = to_pascal_case(table_snake_case)
    if has_dentity:
        cursor.execute(f"SET IDENTITY_INSERT dbo.{table} ON")
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    for entry in tuples:
        query  = "insert into dbo.%s(%s) values (" % (table, cols)
        num = 1
        entry_list = []
        for e in entry:
            if num == len(entry):
                query += "?"
            else:
                query += "?, "
            num += 1
#             print(e)
            if (not isinstance(e, str)) and (not isinstance(e, bool)) and (not (e is None)) and (not (type(e) is pd.Timestamp)) and math.isnan(e):
                entry_list.append(None)
            else:
                entry_list.append(e)
        query += ")"
#         print(query)
#         print(*entry_list)
        cursor.execute(query, *entry_list)
    if has_dentity:
        cursor.execute(f"SET IDENTITY_INSERT dbo.{table} OFF")
    conn.commit()

To delete:
    1) Select Tables
    2) Press F7
    3) Select tables
    4) Delete them till it's done

<h1>Должности, отделы, исполнители старое</h1>

In [275]:
df = pd.DataFrame()

In [276]:
df.insert(0, 'id', range(1, 101))

In [277]:
df.insert(0, 'name', 'Position')

In [278]:
cursor.execute(open("sql_server/25.sql", "r").read())
conn.commit()

In [279]:
execute_values(df, "positions", False)

In [280]:
df = pd.DataFrame()

In [281]:
df.insert(0, 'id', range(1, 27))

In [282]:
df.insert(0, 'code', list(map(chr, range(97, 123))))

In [283]:
df.insert(0, 'name', 'Department')

In [284]:
df.insert(0, 'short_name', 'Pos')

In [285]:
df.insert(0, 'is_active', True)

In [286]:
df.loc[:, ["id", "code"]].to_csv("departments.csv")

In [287]:
cursor.execute(open("sql_server/30.sql", "r").read())
conn.commit()

In [288]:
execute_values(df, "departments")

In [289]:
df = pd.DataFrame()

In [290]:
df.insert(0, 'id', range(1, 464))

In [291]:
df.insert(0, 'department_id', np.random.randint(26, size=463) + 1)

In [292]:
df.insert(0, 'position_id', np.random.randint(100, size=463) + 1)

In [293]:
df.insert(0, 'fullname', 'Name Surname Patronymic')

In [294]:
df.insert(0, 'name', 'Name')

In [295]:
df.insert(0, 'is_active', True)

In [296]:
df["id"].to_csv("employee_ids.csv")

In [297]:
cursor.execute(open("sql_server/26.sql", "r").read())
conn.commit()

In [298]:
execute_values(df, "employees", False)

<h1>Должности</h1>

In [173]:
df = pd.read_json('post.json')

In [174]:
df["name"] = df["name"].str.strip()

In [633]:
cursor.execute(open("sql_server/25.sql", "r").read())
conn.commit()

In [176]:
execute_values(df, "positions", False)

<h1>Отделы</h1>

In [711]:
df = pd.read_json('department.json')

In [712]:
df["name"] = df["name"].str.strip()
df["reduction"] = df["reduction"].str.strip()

In [180]:
df = df.rename(columns={"id" : "code",
                        "reduction": "short_name",
                        "enable": "is_active"
                       })

In [181]:
df.insert(0, 'id', range(1, len(df) + 1))

In [182]:
df.loc[:, ["id", "code"]].to_csv("departments.csv")

In [632]:
cursor.execute(open("sql_server/30.sql", "r").read())
conn.commit()

In [184]:
execute_values(df, "departments")

<h1>Исполнители</h1>

In [185]:
df = pd.read_json('staff.json')

In [186]:
df_dep = df['department'].apply(pd.Series)

In [187]:
df_dep = df_dep.drop(["name", "reduction", "enable"], axis=1)

In [188]:
df_dep = df_dep.rename(columns={"id": "code"})

In [189]:
dep = pd.read_csv("departments.csv")
dep = dep.drop(["Unnamed: 0"], axis=1)

In [190]:
for index, row in df_dep.iterrows():
    try:
        df_dep.loc[index, "department_id"] = dep.loc[dep["code"] == row["code"], "id"].values[0]
    except:
        df_dep.loc[index, "department_id"] = None

In [191]:
df_dep = df_dep.drop(["code"], axis=1)

In [192]:
df["department_id"] = df_dep

In [193]:
df_post = df['post'].apply(pd.Series)

In [194]:
df_post = df_post.drop(["name"], axis=1)

In [195]:
df_post = df_post.rename(columns={"id": "position_id"})

In [196]:
df["position_id"] = df_post

In [197]:
df["is_active"] = df["dismissedDate"].isna()

In [198]:
df = df.drop(["tableNumber", "userName", "createdDate", "dismissedDate", "vacation", "post", "department"], axis=1)

In [199]:
df = df.dropna()

In [200]:
df["name"] = df["fullName"].str.partition().loc[:, 0]

In [202]:
df["id"].to_csv("employee_ids.csv")

In [ ]:
cursor.execute(open("sql_server/26.sql", "r").read())
conn.commit()

In [204]:
execute_values(df, "employees", False)

<h1>Пользователи</h1>

In [299]:
df = pd.DataFrame({'id': [1, 2], 'login': ["1", "2"], 'password': ["1", "2"], 'employee_id': [5, 6]})

In [300]:
cursor.execute(open("sql_server/xx_users.sql", "r").read())
conn.commit()

In [301]:
execute_values(df, "users")

<h1>Значения по умолчанию</h1>

In [302]:
df = pd.DataFrame({'user_id': [1, 2], 'department_id': [None, None], 'creator_id': [5, 6], 'inspector_id': [None, None], 'norm_contr_id': [None, None]})

In [303]:
cursor.execute(open("sql_server/xx_default.sql", "r").read())
conn.commit()

In [304]:
execute_values(df, "default_values", False)

<h1>Условия эксплуатации</h1>

In [305]:
df1 = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["неагрессивная", "слабоагрессивная", "среднеагрессивная", "сильноагрессивная"]})

In [306]:
df2 = pd.DataFrame({'id': [1, 2, 3], 'name': ["внутри помещения", "снаружи помещения", "в жидкостях"]})

In [307]:
df3 = pd.DataFrame({'id': [1, 2, 3, 4, 5], 'name': ["нет газов (вода)", "газы группы А (кислоты)", "газы группы B, C и D (щелочи)", "бензин", "масло"]})

In [308]:
df4 = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["сталь без покрытия", "оцинкованная сталь", "сталь горячего цинкования", "сталь с газотермическим напылением цинка"]})

In [309]:
df5 = pd.DataFrame({'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 'name': ["ПФ", "АУ", "ЭЭ", "МА", "МБ", "НЦ", "ФФ", "ПА", "ПЭ", "ПБ", "ПВ", "СВ", "ПУ", "ЭП", "ПС", "КО", "АЦ"]})

In [310]:
df6 = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["отсутствуют", "сдвигоустойчивые соединения", "фланцевые соединения", "сдвигоустойчивые и фланцевые соединения"]})

In [311]:
df5.loc[:, ["id", "name"]].to_csv("paintwork.csv")

In [312]:
cursor.execute(open("sql_server/y_1.sql", "r").read())
cursor.execute(open("sql_server/y_2.sql", "r").read())
cursor.execute(open("sql_server/y_3.sql", "r").read())
cursor.execute(open("sql_server/y_4.sql", "r").read())
cursor.execute(open("sql_server/y_5.sql", "r").read())
cursor.execute(open("sql_server/y_6.sql", "r").read())
conn.commit()

In [ ]:
execute_values(df1, "env_aggressiveness")

In [ ]:
execute_values(df2, "operating_areas")

In [ ]:
execute_values(df3, "gas_groups")

In [ ]:
execute_values(df4, "construction_materials")

In [316]:
execute_values(df5, "paintwork_types")

In [ ]:
execute_values(df6, "high_tensile_bolts_types")

<h1>Стойкость лакокрасочных покрытий</h1>

In [320]:
df = pd.DataFrame({'id': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'name': ["П", "А", "Х", "В", "Б", "М", "ХЩ", "ХК", "Т"]})

In [321]:
df.loc[:, ["id", "name"]].to_csv("paintwork_fastness.csv")

In [322]:
cursor.execute(open("sql_server/y_7.sql", "r").read())
conn.commit()

In [ ]:
execute_values(df, "paintwork_fastness")

<h1>Контроль сварки</h1>

In [224]:
df = pd.DataFrame({'Id': [1, 2, 3, 4], 'Name': ["не требуется", "опрыскиванием керосином", "ультразвуком", "гамма-излучением"]})

In [225]:
cursor.execute(open("sql_server/yy_1.sql", "r").read())
conn.commit()

In [226]:
execute_values(df, "WeldingControl")

<h1>Типы документов</h1>

In [227]:
df = pd.DataFrame({'id': [1, 2, 3, 4, 5], 'code': ["л.", "СМ", "ВМП", "СМС", "РР"], 'name': ["Листы основного комплекта", "Спецификация металлопроката", "Ведомость металлоконструкций по видам профилей", "Сводная спецификация металлопроката", "Расчет металлоконструкций"]})

In [228]:
cursor.execute(open("sql_server/x.sql", "r").read())
conn.commit()

In [229]:
execute_values(df, "doc_types")

<h1>Грунтовка</h1>

In [230]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Грунт_аз', db_conn)

In [231]:
df = df.replace("", np.nan)

In [232]:
df["Приоритет"] = df["Приоритет"].map(lambda x: 0 if pd.isna(x) else x)

In [233]:
df = df.rename(columns={"поз": "id",
                        "Группа_гр": "group_num",
                        "Наим_гр": "name",
                        "Можно_гр": "can_be_primed",
                        "Приоритет": "priority"
                       })

In [234]:
cursor.execute(open("sql_server/4.sql", "r").read())
conn.commit()

In [235]:
execute_values(df, "primer")

<h1>Степени очистки антикоррозионной защиты</h1>

In [236]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Степени_оч_аз', db_conn)

In [237]:
df["Наим_оч"] = df["Наим_оч"].str.strip()

In [238]:
df = df.rename(columns={"Степень_оч": "id",
                        "Наим_оч": "name"
                       })

In [239]:
cursor.execute(open("sql_server/44.sql", "r").read())
conn.commit()

In [240]:
execute_values(df, "corr_prot_cleaning_degrees")

<h1>Варианты антикоррозионной защиты </h1>

In [241]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Варианты_аз', db_conn)

In [242]:
df["Тип_лп"] = df["Тип_лп"].str.strip()
df["Стойкость_лп"] = df["Стойкость_лп"].str.strip()

In [243]:
df = df.replace("", np.nan)

In [244]:
df = df.drop_duplicates()

In [245]:
df = df.drop_duplicates(subset=["Зона_экс", "Группа_газов", "Агресс_ср", "Матер_к", "Тип_лп", "Группа_лп", "Стойкость_лп"])

In [246]:
paintwork = pd.read_csv("paintwork.csv")
paintwork = paintwork.drop(["Unnamed: 0"], axis=1)

In [247]:
for index, row in df.iterrows():
    try:
        df.loc[index, "paintwork_type_id"] = paintwork.loc[paintwork["name"] == row["Тип_лп"], "id"].values[0]
    except:
        df.loc[index, "paintwork_type_id"] = None

In [248]:
paintwork_fastness = pd.read_csv("paintwork_fastness.csv")
paintwork_fastness = paintwork_fastness.drop(["Unnamed: 0"], axis=1)

In [249]:
for index, row in df.iterrows():
    try:
        df.loc[index, "paintwork_fastness_id"] = paintwork_fastness.loc[paintwork_fastness["name"] == row["Стойкость_лп"], "id"].values[0]
    except:
        df.loc[index, "paintwork_fastness_id"] = None

In [250]:
df = df.drop(["Тип_лп", "Стойкость_лп"], axis=1)

In [251]:
df["Группа_газов"] += 1
df["Агресс_ср"] += 1
df["Матер_к"] += 1

In [252]:
df = df.rename(columns={"Вариант": "id",
                        "Зона_экс": "operating_area_id",
                        "Группа_газов": "gas_group_id",
                        "Агресс_ср": "env_aggressiveness_id",
                        "Матер_к": "construction_material_id",
                        "Группа_лп": "paintwork_group",
                        "Кол_слоев_лп": "paintwork_num_of_layers",
                        "Толщ_лпгр": "paintwork_primer_thickness",
                        "Кол_слоев_гр": "primer_num_of_layers",
                        "Степень_оч": "cleaning_degree_id",
                        "Статус_аз": "status"
                       })

In [253]:
df = df.where(pd.notnull(df), None)

In [254]:
cursor.execute(open("sql_server/1.sql", "r").read())
conn.commit()

In [255]:
execute_values(df, "corr_prot_variants")

<h1>Способы антикоррозионной защиты</h1>

In [256]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Способы_аз', db_conn)

In [257]:
df["Наим_сп"] = df["Наим_сп"].str.strip()

In [258]:
df["Агресс_ср"] += 1
df["Матер_к"] += 1

In [259]:
df.insert(0, 'id', range(1, len(df) + 1))

In [260]:
df = df.rename(columns={"Агресс_ср": "env_aggressiveness_id",
                        "Матер_к": "construction_material_id",
                        "Наим_сп": "name",
                        "Статус_сп": "status"
                       })

In [261]:
cursor.execute(open("sql_server/19.sql", "r").read())
conn.commit()

In [262]:
execute_values(df, "corr_prot_methods")

<h1>Покрытия антикоррозионной защиты</h1>

In [263]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Покрытия_аз', db_conn)

In [264]:
df = df.replace("", None)

In [265]:
df["Тип_лп"] = df["Тип_лп"].str.strip()
df["Стойкость_лп"] = df["Стойкость_лп"].str.strip()
df["Наим_лп"] = df["Наим_лп"].str.strip()

In [266]:
df["Приоритет"] = df["Приоритет"].map(lambda x: 0 if pd.isna(x) else x)

In [267]:
df = df.drop_duplicates(subset=["Тип_лп", "Группа_лп", "Стойкость_лп", "Наим_лп"])

In [268]:
paintwork = pd.read_csv("paintwork.csv")
paintwork = paintwork.drop(["Unnamed: 0"], axis=1)

In [269]:
for index, row in df.iterrows():
    try:
        df.loc[index, "paintwork_type_id"] = paintwork.loc[paintwork["name"] == row["Тип_лп"], "id"].values[0]
    except:
        df.loc[index, "paintwork_type_id"] = None

In [270]:
paintwork_fastness = pd.read_csv("paintwork_fastness.csv")
paintwork_fastness = paintwork_fastness.drop(["Unnamed: 0"], axis=1)

In [271]:
for index, row in df.iterrows():
    try:
        df.loc[index, "paintwork_fastness_id"] = paintwork_fastness.loc[paintwork_fastness["name"] == row["Стойкость_лп"], "id"].values[0]
    except:
        df.loc[index, "paintwork_fastness_id"] = None

In [272]:
df = df.drop(["Тип_лп", "Стойкость_лп"], axis=1)

In [273]:
df = df.rename(columns={"Покрытие": "id",
                        "Группа_лп": "paintwork_group",
                        "Наим_лп": "name",
                        "Кол_сл_лп": "paintwork_num_of_layers",
                        "Группа_гр": "primer_group",
                        "Можно_лп": "can_be_painted",
                        "Приоритет": "priority"
                       })

In [274]:
df = df.where(pd.notnull(df), None)

In [275]:
cursor.execute(open("sql_server/17.sql", "r").read())
conn.commit()

In [276]:
execute_values(df, "corr_prot_coatings")

<h1>Виды конструкций</h1>

In [326]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_видов_к', db_conn)

In [327]:
df["Имя_к"] = df["Имя_к"].str.strip()

In [328]:
df = df[df["Имя_к"].isna() == False]

In [329]:
df = df.rename(columns={"Шифр_к": "id",
                        "Имя_к": "name"
                       })

In [330]:
df.loc[df["id"] == 2, "name"] = "Двухветвевые колонны одноэтажных производственных зданий и крановых эстакад"
df.loc[df["id"] == 9, "name"] = "Подкраново-подстропильные фермы"
df.loc[df["id"] == 34, "name"] = "Переплеты, витражи из профильных труб с деталями крепления"

In [331]:
cursor.execute(open("sql_server/22.sql", "r").read())
conn.commit()

In [332]:
execute_values(df, "construction_types")

<h1>Подвиды конструкций</h1>

In [333]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_видов_пк', db_conn)

In [334]:
df["Имя_пк"] = df["Имя_пк"].str.strip()
df["Расценка"] = df["Расценка"].str.strip()

In [335]:
df = df.drop_duplicates()

In [336]:
df = df.drop(["Шифр_пк"], axis=1)

In [337]:
df = df.rename(columns={"Поз_пк": "id",
                        "Шифр_к": "type_id",
                        "Имя_пк": "name",
                        "Расценка": "valuation"
                       })

In [338]:
df.loc[df["id"] == 11, "name"] = "высотой свыше 150 м, поставляемые отдельными элементами"
df.loc[df["id"] == 14, "name"] = "коробчатого сечения пролетом 12 м и более"
df.loc[df["id"] == 33, "name"] = "с шагом стропильных ферм 6 м"
df.loc[df["id"] == 34, "name"] = "с шагом стропильных ферм 12 м"
df.loc[df["id"] == 49, "name"] = "составного сечения из горячекатаных профилей, соединенных планками"
df.loc[df["id"] == 51, "name"] = "ветви из двутавров с парал. гранями полок, решетка из г/кат. уголков и швеллеров"
df.loc[df["id"] == 52, "name"] = "ветви из двутавров (ГОСТ 8239-72) или швеллеров и решетка из г/кат. уголков"
df.loc[df["id"] == 69, "name"] = "с поясами из тавров, стенкой из листовой стали, пролетом 6 м, массой 1 м,т:"
df.loc[df["id"] == 70, "name"] = "с поясами из тавров, стенкой из листовой стали, пролетом 12 м и более,т:"
df.loc[df["id"] == 71, "name"] = "из листовой стали пролетом 6 м, массой 1 м,т:"
df.loc[df["id"] == 72, "name"] = "из листовой стали пролетом 12 м, массой 1 м,т:"
df.loc[df["id"] == 83, "name"] = "пролетом 12 м, решетчатые из г/кат. и гнутых профилей, масса 1 м до 0.4 т"
df.loc[df["id"] == 84, "name"] = "пролетом 12 м, решетчатые из г/кат. и гнутых профилей, масса 1 м свыше 0.4 т"
df.loc[df["id"] == 85, "name"] = "пролетом 12 м, решетчатые сост. сечения из листовой стали, масса 1 м до 0.4 т"
df.loc[df["id"] == 86, "name"] = "пролетом 12 м, решетчатые сост. сечения из листовой стали, масса 1 м свыше 0.4 т"
df.loc[df["id"] == 87, "name"] = "составного сечения из листовой стали, масса 1 м,т:"
df.loc[df["id"] == 88, "name"] = "составного сечения из двух и более уголков или швеллеров, масса 1 м,т:"
df.loc[df["id"] == 89, "name"] = "из двутавров с параллельными гранями полок, масса 1 м,т:"
df.loc[df["id"] == 90, "name"] = "составного сечения из листовой стали с верхним опиранием настила, масса марки,т:"
df.loc[df["id"] == 91, "name"] = "составного сечения при пониженном опирании настила, масса марки,т:"
df.loc[df["id"] == 92, "name"] = "из двутавров с парал. гранями полок с верхним опиранием настила, масса до 1 т"
df.loc[df["id"] == 93, "name"] = "из двутавров с парал. гранями полок с верхним опиранием настила, масса свыше 1 т"
df.loc[df["id"] == 94, "name"] = "из двутавров с парал. гранями полок с пониж. опиранием настила, масса до 1 т"
df.loc[df["id"] == 95, "name"] = "из двутавров с парал. гранями полок с пониж. опиранием настила, масса свыше 1 т"
df.loc[df["id"] == 113, "name"] = "фермы из парных уголков и тавров, попер. балки из двутавров, масса 1 м до 0.5 т"
df.loc[df["id"] == 114, "name"] = "фермы из парных уголков и тавров, попер. балки из двутавров, масса 1 м до 1 т"
df.loc[df["id"] == 115, "name"] = "фермы из парных уголков и тавров, попер. балки из двутавров, масса 1 м свыше 1 т"

In [339]:
cursor.execute(open("sql_server/24.sql", "r").read())
conn.commit()

In [340]:
execute_values(df, "construction_subtypes")

<h1>Типовые наименования листов</h1>

In [341]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_листов_н', db_conn)

In [342]:
df["ш_листа"] = df["ш_листа"].str.strip()

In [343]:
df = df.rename(columns={"ш_номер": "id",
                        "ш_листа": "name"
                       })

In [344]:
cursor.execute(open("sql_server/27.sql", "r").read())
conn.commit()

In [345]:
execute_values(df, "sheet_names")

<h1>Виды ссылочных документов</h1>

In [346]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_ссыл_в', db_conn)

In [347]:
df["Наим_вида"] = df["Наим_вида"].str.strip()
df["Наим_вида"] = df["Наим_вида"].str.capitalize()

In [348]:
df = df.drop_duplicates(subset=["Наим_вида"])

In [349]:
df = df.rename(columns={"Вид_сд": "id",
                        "Наим_вида": "name"
                       })

In [350]:
cursor.execute(open("sql_server/40.sql", "r").read())
conn.commit()

In [351]:
execute_values(df, "linked_doc_types")

<h1>Ссылочные документы</h1>

In [352]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_ссыл', db_conn)

In [353]:
df = df.replace("", np.nan)

In [354]:
df = df.dropna()

In [355]:
df = df.drop_duplicates(subset=["Шифр_сд"])

In [356]:
df = df.reset_index(drop=True)

In [357]:
df["Шифр_сд"] = df["Шифр_сд"].str.strip()
df["Обозн_сд"] = df["Обозн_сд"].str.strip()
df["Наим_сд"] = df["Наим_сд"].str.strip()

In [358]:
for i in range(0, len(df)):
    arr = df["Наим_сд"][i].split()
    new_str = ' '.join(arr)
    df.at[i, "Наим_сд"] = new_str

In [359]:
df["Наим_сд"] = df["Наим_сд"].str.replace("возду ха", "воздуха")
df["Наим_сд"] = df["Наим_сд"].str.replace("Материриалы", "Материалы")
df["Наим_сд"] = df["Наим_сд"].str.replace("рекоментации", "рекомендации")
df["Наим_сд"] = df["Наим_сд"].str.replace("рас пашные", "распашные")
df["Наим_сд"] = df["Наим_сд"].str.replace("металличе ческих", "металлических")
df["Наим_сд"] = df["Наим_сд"].str.replace("металличе ских", "металлических")
df["Наим_сд"] = df["Наим_сд"].str.replace("электри ческие", "электрические")
df["Наим_сд"] = df["Наим_сд"].str.replace("кра ны", "краны")
df["Наим_сд"] = df["Наим_сд"].str.replace("примене нием", "применением")
df["Наим_сд"] = df["Наим_сд"].str.replace("обслуживани сосудов", "обслуживания сосудов")
df["Наим_сд"] = df["Наим_сд"].str.replace("пе реплетами", "переплетами")
df["Наим_сд"] = df["Наим_сд"].str.replace("за полнением", "заполнением")
df["Наим_сд"] = df["Наим_сд"].str.replace("линй", "линий")
df["Наим_сд"] = df["Наим_сд"].str.replace("мосто вые", "мостовые")
df["Наим_сд"] = df["Наим_сд"].str.replace("покрыти ем", "покрытием")
df["Наим_сд"] = df["Наим_сд"].str.replace("со кращенному", "сокращенному")
df["Наим_сд"] = df["Наим_сд"].str.replace("покрытияпролетами", "покрытия пролетами")
df["Наим_сд"] = df["Наим_сд"].str.replace("широкополочныых", "широкополочных")
df["Наим_сд"] = df["Наим_сд"].str.replace("тав ров", "тавров")
df["Наим_сд"] = df["Наим_сд"].str.replace("зда ний", "зданий")
df["Наим_сд"] = df["Наим_сд"].str.replace("ремонтно -монтажных", "ремонтно-монтажных")
df["Наим_сд"] = df["Наим_сд"].str.replace("пок рытия", "покрытия")
df["Наим_сд"] = df["Наим_сд"].str.replace("профилирован ным", "профилированным")
df["Наим_сд"] = df["Наим_сд"].str.replace("стеклои минераловатных", "стекло и минераловатных")
df["Наим_сд"] = df["Наим_сд"].str.replace("окон,дверей", "окон, дверей")
df["Наим_сд"] = df["Наим_сд"].str.replace("пром.зданий", "пром. зданий")
df["Наим_сд"] = df["Наим_сд"].str.replace("строитель ные", "строительные")
df["Наим_сд"] = df["Наим_сд"].str.replace("сталь ные", "стальные")
df["Наим_сд"] = df["Наим_сд"].str.replace("наруж ного", "наружного")
df["Наим_сд"] = df["Наим_сд"].str.replace("ветров.район.", "ветров. район.")
df["Наим_сд"] = df["Наим_сд"].str.replace("выше и ниже -40 град.с.  ше и ниже -40 град.с.", "выше -40 град.с. и ниже -40 град.с.")
df["Наим_сд"] = df["Наим_сд"].str.replace("алю миниевых", "алюминиевых")
df["Наим_сд"] = df["Наим_сд"].str.replace("зданий,промышленных", "зданий, промышленных")
df["Наим_сд"] = df["Наим_сд"].str.replace("предприятий.шарнирные", "предприятий. Шарнирные")
df["Наим_сд"] = df["Наим_сд"].str.replace("связей.Чертежи", "связей. Чертежи")
df["Наим_сд"] = df["Наим_сд"].str.replace("сталь ными", "стальными")
df["Наим_сд"] = df["Наим_сд"].str.replace("унифици рованных", "унифицированных")
df["Наим_сд"] = df["Наим_сд"].str.replace("уз лы", "узлы")
df["Наим_сд"] = df["Наим_сд"].str.replace("железобе тонных", "железобетонных")
df["Наим_сд"] = df["Наим_сд"].str.replace("метал лопроката", "металлопроката")
df["Наим_сд"] = df["Наим_сд"].str.replace("выпу ску", "выпуску")
df["Наим_сд"] = df["Наим_сд"].str.replace("откры вающиеся", "открывающиеся")
df["Наим_сд"] = df["Наим_сд"].str.replace("светоаэра ционных", "светоаэрационных")
df["Наим_сд"] = df["Наим_сд"].str.replace("проектирования.Рабочие", "проектирования. Рабочие")
df["Наим_сд"] = df["Наим_сд"].str.replace("черте жи", "чертежи")
df["Наим_сд"] = df["Наим_сд"].str.replace("опорн.кранами", "опорн. кранами")
df["Наим_сд"] = df["Наим_сд"].str.replace("опорынми кранами", "опорными кранами")
df["Наим_сд"] = df["Наим_сд"].str.replace("несущ.конструкций", "несущ. конструкций")
df["Наим_сд"] = df["Наим_сд"].str.replace("подв.кранами", "подв. кранами")
df["Наим_сд"] = df["Наим_сд"].str.replace("кровлей,оборудованных", "кровлей, оборудованных")
df["Наим_сд"] = df["Наим_сд"].str.replace("одноэт.произв", "одноэт. произв")
df["Наим_сд"] = df["Наим_сд"].str.replace("произв.зданий", "произв. зданий")
df["Наим_сд"] = df["Наим_сд"].str.replace("кровлей,оборудованных", "кровлей, оборудованных")
df["Наим_сд"] = df["Наим_сд"].str.replace("специальн.назначения", "специальн. назначения")
df["Наим_сд"] = df["Наим_сд"].str.replace("зданий,оборудованных", "зданий, оборудованных")
df["Наим_сд"] = df["Наим_сд"].str.replace("две рей", "дверей")

In [360]:
df = df.drop_duplicates(subset=["Наим_сд"])

In [361]:
df.loc[df["Наим_сд"].str.endswith('.') == False, "Наим_сд"] = df[df["Наим_сд"].str.endswith('.') == False]["Наим_сд"] + '.'

In [362]:
df.insert(0, 'id', range(1, len(df) + 1))

In [363]:
df = df.rename(columns={"Шифр_сд": "code",
                        "Вид_сд": "type_id",
                        "Обозн_сд": "designation",
                        "Наим_сд": "name"
                       })

In [364]:
cursor.execute(open("sql_server/39.sql", "r").read())
conn.commit()

In [ ]:
execute_values(df, "linked_docs")

<h1>Категории производства по пожарной и взрывопожарной опасности</h1>

In [386]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Категории_пр', db_conn)

In [387]:
df = df.replace("", np.nan)

In [388]:
df = df.rename(columns={"поз": "id",
                        "Категория": "category",
                        "Название": "name",
                        "Текст_од": "description"
                       })

In [389]:
df = df.where(pd.notnull(df), None)

In [384]:
cursor.execute(open("sql_server/5.sql", "r").read())
conn.commit()

In [393]:
execute_values(df, "fire_hazard_categories")

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]String or binary data would be truncated in table 'DocumentsKM.dbo.FireHazardCategories', column 'Category'. Truncated value: ''. (2628) (SQLExecDirectW); [42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")

<h1>Проекты старые</h1>

In [399]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Марки', db_conn)

df = df.dropna(subset=["код_марки", "обозн", "комплекс", "объект"])

e_ids = pd.read_csv("employee_ids.csv")
e_ids = e_ids.drop(["Unnamed: 0"], axis=1)
e_ids = e_ids.values.flatten().tolist()
df.loc[df["гл_спец"].isin(e_ids) == False, "гл_спец"] = None
df.loc[df["рук_гр"].isin(e_ids) == False, "рук_гр"] = None
df.loc[df["н_контр"].isin(e_ids) == False, "н_контр"] = None

df = df[df["обозн"].str.contains("демонтаж") == False]
df = df[df["обозн"].str.contains("Мновый") == False]
df["обозн"] = df["обозн"].str.replace(",", ".")

df2 = pd.DataFrame(df["обозн"].str.split('-КМ', 1).tolist(),
                                 columns = ['first','code'])

df = df.reset_index()
df2 = df2.join(df)

df2 = df2.dropna(subset=["code"])

df2["code"] = "КМ" + df2["code"]

df3 = pd.DataFrame(df2["first"].str.split('.', 1).tolist(),
                                 columns = ['baseSeries','middle'])

df2 = df2.reset_index()
df3 = df3.reset_index()
df4 = df2.join(df3[["baseSeries", "middle"]])

df4 = df4.drop(["first", "index"], axis=1)
df4 = df4.rename(columns={"код_марки": "id"})

df5 = pd.DataFrame(df4["middle"].fillna("").str.split('.', 1).tolist(),
                                 columns = ['node','subnode'])

df6 = pd.DataFrame(df5["subnode"].fillna("").str.split('-', 1).tolist(),
                                 columns = ['subnode_part','node_part'])

df5 = df5.reset_index()
df6 = df6.reset_index()
df7 = df5.join(df6[["subnode_part", "node_part"]])

df7["node_part"] = "-" + df7["node_part"]
df7["node2"] = df7["node"] + df7["node_part"].fillna("")

df4 = df4.reset_index(drop=True)
df7 = df7.reset_index(drop=True)
df8 = df4.join(df7[["node2", "subnode_part"]])

df8 = df8.rename(columns={"node2": "nodeCode", "subnode_part": "subnodeCode"})
df8 = df8.drop(["middle"], axis=1)

enc = OrdinalEncoder()
res = enc.fit_transform(np.array(df8["baseSeries"]).reshape(-1, 1))

s = pd.DataFrame(pd.Series(res.flatten()) + 1)
s = s.rename(columns={0: "project_id"})

df9 = df8.join(s)

projects_df = df9[["baseSeries", "комплекс", "project_id"]]

projects_df = projects_df.drop_duplicates(subset=["project_id"])

projects_df = projects_df.rename(columns={"baseSeries": "base_series", "комплекс": "name", "project_id": "id"})

cursor.execute(open("sql_server/xx_projects.sql", "r").read())
conn.commit()

execute_values(projects_df, "projects", False)

df10 = pd.DataFrame(df9["объект"].str.split('.', 1).tolist(),
                                 columns = ['f_p','o_p'])

df10 = df10.fillna("")

df11 = df9.join(df10)

df11["node_feature"] = df11["baseSeries"] + df11["nodeCode"]

enc = OrdinalEncoder()
res = enc.fit_transform(np.array(df11["node_feature"]).reshape(-1, 1))

s = pd.DataFrame(pd.Series(res.flatten()) + 1)
s = s.rename(columns={0: "node_id"})

df12 = df11.join(s)

nodes_df = df12[["node_id", "f_p", "project_id", "nodeCode"]]

chiefEngineerIds = [1, 2, 3]

nodes_df["chief_engineer"] = random.choices(chiefEngineerIds, k=len(nodes_df))

nodes_df = nodes_df.rename(columns={"node_id": "id", "f_p": "name", "nodeCode": "code", "chief_engineer": "chief_engineer_id"})

nodes_df = nodes_df.drop_duplicates(subset=["id"])

cursor.execute(open("sql_server/xx_nodes.sql", "r").read())
conn.commit()

execute_values(nodes_df, "nodes", False)

df13 = pd.DataFrame(df12["o_p"].str.split('.', 1).tolist(),
                                 columns = ['s_p','m_p'])

df13 = df13.fillna("")

df13["s_p"] = df13["s_p"].str.strip()
df13["m_p"] = df13["m_p"].str.strip()

df14 = df12.join(df13)

df14["subnode_feature"] = df11["node_feature"] + df11["subnodeCode"]

enc = OrdinalEncoder()
res = enc.fit_transform(np.array(df14["subnode_feature"]).reshape(-1, 1))

s = pd.DataFrame(pd.Series(res.flatten()) + 1)
s = s.rename(columns={0: "subnode_id"})

df15 = df14.join(s)

subnodes_df = df15[["subnode_id", "node_id", "subnodeCode", "s_p"]]

subnodes_df = subnodes_df.rename(columns={"subnode_id": "id", "s_p": "name", "subnodeCode": "code"})

subnodes_df = subnodes_df.drop_duplicates(subset=["id"])

cursor.execute(open("sql_server/xx_subnodes.sql", "r").read())
conn.commit()

execute_values(subnodes_df, "subnodes", False)

df15 = df15.drop(["обозн", "комплекс", "объект", "level_0", "ОЗ_марки"], axis=1)

df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("минус ", "-")
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("плюс ", "")
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("+", "")
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("- 39", "-39")
df15 = df15[df15["t_эксплуат"] != "-"]
df15 = df15[df15["t_эксплуат"] != "минус"]
df15["t_эксплуат"] = df15["t_эксплуат"].astype("float64")

df15["об_марки"] = df15["об_марки"].str.strip()
df15["об_марки"] = df15["об_марки"].str.replace("KM", "КМ")
df15 = df15[df15["об_марки"] != "по"]

df15 = df15.drop(["об_марки"], axis=1)

df15 = df15.drop(["project_id", "f_p", "o_p", "s_p", "subnode_feature", "node_feature", "subnodeCode"], axis=1)

df15 = df15.drop(["baseSeries", "nodeCode", "node_id"], axis=1)

df15["прим"] = df15["прим"].str.strip()
df15["прим"] = df15["прим"].str.replace("\r\n       ", " ")

df15 = df15.drop(["ДопКод"], axis=1)

df15["категория_пр"] = 3

df15["агрессивность"] += 1
df15["группа_газов"] += 1
df15["материал"] += 1
df15["вп_болты"] += 1

# Part
df15.loc[df15["тип_ЛКМ"] == "ПФ", "тип_ЛКМ"] = 1
df15.loc[df15["тип_ЛКМ"] == "АУ", "тип_ЛКМ"] = 2
df15.loc[df15["тип_ЛКМ"] == "ЭЭ", "тип_ЛКМ"] = 3
df15.loc[df15["тип_ЛКМ"] == "МА", "тип_ЛКМ"] = 4
df15.loc[df15["тип_ЛКМ"] == "ПВ", "тип_ЛКМ"] = 11

df15 = df15.drop_duplicates(subset=["code", "subnode_id"])

df15_operating_cond = df15.copy()

df15 = df15.rename(columns={"код_отд": "department_id",
                            "подп1": "signed1_id",
                            "подп2": "signed2_id",
                            "гл_спец": "chief_specialist_id",
                            "рук_гр": "group_leader_id",
                            "н_контр": "norm_contr_id",
                            "дата_выд": "issue_date",
                            "кол_томов": "num_of_volumes",
                            "дата_ред": "edited_date",
                            "коэф_надежн": "safety_coeff",
                            "t_эксплуат": "operating_temp",
                            "зона_эксплуат": "operating_zone",
                            "группа_газов": "gas_group",
                            "агрессивность": "aggressiveness",
                            "материал": "material",
                            "тип_ЛКМ": "paintwork_type",
                            "прим": "note",
                            "категория_пр": "fire_hazard_category_id",
                            "вп_болты": "high_tensile_bolts",
                            "п_транспорт": "p_transport",
                            "п_площадки": "p_site",
                            "10ХСНД": "xcnd",
                            "ТекстЗдСм": "task_text",
                            "ДопОбъемы": "additional_text",
                            "ВесПоВМП": "vmp_weight",
                            "ВыпЗдСм": "impl_3d_estimate",
                            "m_p": "name"
                       })

df15["issue_date"] = df15["issue_date"].astype(object).where(df15["issue_date"].notnull(), None)
df15["edited_date"] = df15["edited_date"].astype(object).where(df15["edited_date"].notnull(), None)
df15 = df15.where(pd.notnull(df15), None)

df15["chief_specialist_id"] = df15["chief_specialist_id"].replace(0, None)
df15["group_leader_id"] = df15["group_leader_id"].replace(0, None)
df15["norm_contr_id"] = df15["norm_contr_id"].replace(0, None)

df16 = df15.drop(["vmp_weight", "impl_3d_estimate", "xcnd", "task_text", "additional_text", "safety_coeff", "aggressiveness", "operating_temp", "operating_zone", "gas_group", "material", "high_tensile_bolts"], axis=1)

df16 = df16.reset_index(drop=True)

df16["id"].to_csv("mark_ids.csv")

cursor.execute(open("sql_server/7.sql", "r").read())
conn.commit()

execute_values(df16, "marks")

df_op_cond = df15_operating_cond.loc[:, ["id", "коэф_надежн", "агрессивность", "t_эксплуат", "зона_эксплуат", "группа_газов", "материал", "тип_ЛКМ", "вп_болты"]]

df_op_cond["t_эксплуат"] = df_op_cond["t_эксплуат"].fillna(-34)

df_op_cond = df_op_cond.rename(columns={"id": "mark_id",
                            "коэф_надежн": "safety_coeff",
                            "t_эксплуат": "temperature",
                            "зона_эксплуат": "operating_area_id",
                            "группа_газов": "gas_group_id",
                            "агрессивность": "env_aggressiveness_id",
                            "материал": "construction_material_id",
                            "тип_ЛКМ": "paintwork_type_id",
                            "вп_болты": "high_tensile_bolts_type_id"
                       })

# cursor.execute(open("sql_server/y_mark_operating_conditions.sql", "r").read())
# conn.commit()

# execute_values(df_op_cond, "mark_operating_conditions")

df_task = df15.loc[:, ["id", "task_text", "additional_text"]]

df_task["task_text"] = df_task["task_text"].str.strip()

df['обозн'] = 'Разработать сметную документацию к чертежам ' + df['обозн'].astype(str) + '\r\nСостав и объемы работ:'

df_task.loc[df_task["task_text"].isna(), "task_text"] = list(df.loc[df["код_марки"].isin(df_task.loc[df_task["task_text"].isna(), "id"]), "обозн"])

df_task = df_task.rename(columns={"id": "mark_id"})

df_task["task_text"] = df_task["task_text"].str.replace("\r\n", "\n")

df_task["additional_text"] = df_task["additional_text"].str.replace("\r\n", "\n")

cursor.execute(open("sql_server/y_estimate_task.sql", "r").read())
conn.commit()

execute_values(df_task, "estimate_task")

df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.ВыпСпец', db_conn)

df = df.replace("", np.nan)

df = df.drop_duplicates(subset=["код_марки", "выпуск"])

df = df.drop(["вкл_вып"], axis=1)

df["текущий"] = df["текущий"].map(lambda x: True if (x == 1) else False)

df.insert(0, 'id', range(1, len(df) + 1))

df = df.rename(columns={"код_марки": "mark_id",
                        "выпуск": "num",
                        "дата_созд": "created_date",
                        "текущий": "is_current",
                        "прим": "note"
                       })

df["created_date"] = df["created_date"].astype(object).where(df["created_date"].notnull(), datetime(2000, 1, 1))
df = df.where(pd.notnull(df), None)

m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()

df = df[df["mark_id"].isin(m_ids)]

df.loc[:, ["id", "mark_id", "num"]].to_csv("specs.csv")

cursor.execute(open("sql_server/3.sql", "r").read())
conn.commit()

execute_values(df, "specifications")

C:\Users\sigla\Anaconda3\lib\site-packages\ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ProgrammingError: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Table 'dbo.EstimateTask' does not have the identity property. Cannot perform SET operation. (8106) (SQLExecDirectW)")

<h1>Проекты</h1>

In [724]:
df = pd.read_sql_query('SELECT * FROM ArchDocT.dbo.Проекты', archive_conn)

In [725]:
df = df.loc[:, ["Проект", "Название", "БазСерия"]]

In [726]:
df = df.dropna()

In [727]:
df.loc[:, "Название"] = df["Название"].str.strip()
df.loc[:, "БазСерия"] = df["БазСерия"].str.strip()

In [728]:
df = df.drop_duplicates(subset=["БазСерия"])

In [729]:
df = df.rename(columns={"Проект": "id",
                        "Название": "name",
                        "БазСерия": "base_series"
                       })

In [730]:
df["id"].to_csv("project_ids.csv")

In [696]:
cursor.execute(open("sql_server/xx_projects.sql", "r").read())
conn.commit()

In [697]:
execute_values(df, "projects", False)

In [323]:
# df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Марки', db_conn)

In [324]:
# df = df.dropna(subset=["код_марки", "обозн", "комплекс", "объект"])

In [325]:
# e_ids = pd.read_csv("employee_ids.csv")
# e_ids = e_ids.drop(["Unnamed: 0"], axis=1)
# e_ids = e_ids.values.flatten().tolist()
# df.loc[df["гл_спец"].isin(e_ids) == False, "гл_спец"] = None
# df.loc[df["рук_гр"].isin(e_ids) == False, "рук_гр"] = None
# df.loc[df["н_контр"].isin(e_ids) == False, "н_контр"] = None

In [326]:
# df = df[df["обозн"].str.contains("демонтаж") == False]
# df = df[df["обозн"].str.contains("Мновый") == False]
# df["обозн"] = df["обозн"].str.replace(",", ".")

In [327]:
# df2 = pd.DataFrame(df["обозн"].str.split('-КМ', 1).tolist(),
#                                  columns = ['first','code'])

In [328]:
# df = df.reset_index()
# df2 = df2.join(df)

In [329]:
# df2 = df2.dropna(subset=["code"])

In [330]:
# df2["code"] = "КМ" + df2["code"]

In [331]:
# df3 = pd.DataFrame(df2["first"].str.split('.', 1).tolist(),
#                                  columns = ['baseSeries','middle'])

In [332]:
# df2 = df2.reset_index()
# df3 = df3.reset_index()
# df4 = df2.join(df3[["baseSeries", "middle"]])

In [333]:
# df4 = df4.drop(["first", "index"], axis=1)
# df4 = df4.rename(columns={"код_марки": "id"})

In [334]:
# df5 = pd.DataFrame(df4["middle"].fillna("").str.split('.', 1).tolist(),
#                                  columns = ['node','subnode'])

In [335]:
# df6 = pd.DataFrame(df5["subnode"].fillna("").str.split('-', 1).tolist(),
#                                  columns = ['subnode_part','node_part'])

In [336]:
# df5 = df5.reset_index()
# df6 = df6.reset_index()
# df7 = df5.join(df6[["subnode_part", "node_part"]])

In [337]:
# df7["node_part"] = "-" + df7["node_part"]
# df7["node2"] = df7["node"] + df7["node_part"].fillna("")

In [338]:
# df4 = df4.reset_index(drop=True)
# df7 = df7.reset_index(drop=True)
# df8 = df4.join(df7[["node2", "subnode_part"]])

In [339]:
# df8 = df8.rename(columns={"node2": "nodeCode", "subnode_part": "subnodeCode"})
# df8 = df8.drop(["middle"], axis=1)

In [340]:
# enc = OrdinalEncoder()
# res = enc.fit_transform(np.array(df8["baseSeries"]).reshape(-1, 1))

In [341]:
# s = pd.DataFrame(pd.Series(res.flatten()) + 1)
# s = s.rename(columns={0: "project_id"})

In [342]:
# df9 = df8.join(s)

In [343]:
# projects_df = df9[["baseSeries", "комплекс", "project_id"]]

In [344]:
# projects_df = projects_df.drop_duplicates(subset=["project_id"])

In [345]:
# projects_df = projects_df.rename(columns={"baseSeries": "base_series", "комплекс": "name", "project_id": "id"})

In [639]:
# cursor.execute(open("sql_server/xx_projects.sql", "r").read())
# conn.commit()

In [347]:
# execute_values(projects_df, "projects", False)

<h1>Узлы</h1>

In [702]:
df = pd.read_sql_query('SELECT * FROM ArchDocT.dbo.Узлы', archive_conn)

In [703]:
df = df.loc[:, ["Узел", "Проект", "КодУзла", "НазвУзла", "ГИП"]]

In [704]:
df = df.dropna(subset=["КодУзла"])

In [ ]:
p_ids = pd.read_csv("project_ids.csv")
p_ids = p_ids.drop(["Unnamed: 0"], axis=1)
p_ids = p_ids.values.flatten().tolist()

In [ ]:
df = df[df["Проект"].isin(m_ids)]

In [348]:
# df10 = pd.DataFrame(df9["объект"].str.split('.', 1).tolist(),
#                                  columns = ['f_p','o_p'])

In [349]:
# df10 = df10.fillna("")

In [350]:
# df11 = df9.join(df10)

In [351]:
# df11["node_feature"] = df11["baseSeries"] + df11["nodeCode"]

In [352]:
# enc = OrdinalEncoder()
# res = enc.fit_transform(np.array(df11["node_feature"]).reshape(-1, 1))

In [353]:
# s = pd.DataFrame(pd.Series(res.flatten()) + 1)
# s = s.rename(columns={0: "node_id"})

In [354]:
# df12 = df11.join(s)

In [355]:
# nodes_df = df12[["node_id", "f_p", "project_id", "nodeCode"]]

In [356]:
# chiefEngineerIds = [173, 337]

In [357]:
# nodes_df["chief_engineer"] = random.choices(chiefEngineerIds, k=len(nodes_df))

<ipython-input-357-24cbdb19fe02>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodes_df["chief_engineer"] = random.choices(chiefEngineerIds, k=len(nodes_df))


In [358]:
# nodes_df = nodes_df.rename(columns={"node_id": "id", "f_p": "name", "nodeCode": "code", "chief_engineer": "chief_engineer_id"})

In [359]:
# nodes_df = nodes_df.drop_duplicates(subset=["id"])

In [360]:
# cursor.execute(open("sql_server/xx_nodes.sql", "r").read())
# conn.commit()

In [361]:
# execute_values(nodes_df, "nodes", False)

<h1>Подузлы</h1>

In [708]:
df = pd.read_sql_query('SELECT * FROM ArchDocT.dbo.Подузлы', archive_conn)

In [709]:
df = df.loc[:, ["Подузел", "Узел", "КодПодуз", "НазвПодузла"]]

In [710]:
df.isna().sum()

Подузел            0
Узел               0
КодПодуз           0
НазвПодузла    64876
dtype: int64

In [362]:
# df13 = pd.DataFrame(df12["o_p"].str.split('.', 1).tolist(),
#                                  columns = ['s_p','m_p'])

In [363]:
# df13 = df13.fillna("")

In [364]:
# df13["s_p"] = df13["s_p"].str.strip()
# df13["m_p"] = df13["m_p"].str.strip()

In [365]:
# df14 = df12.join(df13)

In [366]:
# df14["subnode_feature"] = df11["node_feature"] + df11["subnodeCode"]

In [367]:
# enc = OrdinalEncoder()
# res = enc.fit_transform(np.array(df14["subnode_feature"]).reshape(-1, 1))

In [368]:
# s = pd.DataFrame(pd.Series(res.flatten()) + 1)
# s = s.rename(columns={0: "subnode_id"})

In [369]:
# df15 = df14.join(s)

In [370]:
# subnodes_df = df15[["subnode_id", "node_id", "subnodeCode", "s_p"]]

In [371]:
# subnodes_df = subnodes_df.rename(columns={"subnode_id": "id", "s_p": "name", "subnodeCode": "code"})

In [372]:
# subnodes_df = subnodes_df.drop_duplicates(subset=["id"])

In [373]:
# cursor.execute(open("sql_server/xx_subnodes.sql", "r").read())
# conn.commit()

In [374]:
# execute_values(subnodes_df, "subnodes", False)

<h1>Марки</h1>

In [375]:
df15 = df15.drop(["обозн", "комплекс", "объект", "level_0", "ОЗ_марки"], axis=1)

In [376]:
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("минус ", "-")
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("плюс ", "")
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("+", "")
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("- 39", "-39")
df15 = df15[df15["t_эксплуат"] != "-"]
df15 = df15[df15["t_эксплуат"] != "минус"]
df15["t_эксплуат"] = df15["t_эксплуат"].astype("float64")

In [377]:
df15["об_марки"] = df15["об_марки"].str.strip()
df15["об_марки"] = df15["об_марки"].str.replace("KM", "КМ")
df15 = df15[df15["об_марки"] != "по"]

In [378]:
df15 = df15.drop(["об_марки"], axis=1)

In [379]:
df15 = df15.drop(["project_id", "f_p", "o_p", "s_p", "subnode_feature", "node_feature", "subnodeCode"], axis=1)

In [380]:
df15 = df15.drop(["baseSeries", "nodeCode", "node_id"], axis=1)

In [381]:
df15["прим"] = df15["прим"].str.strip()
df15["прим"] = df15["прим"].str.replace("\r\n       ", " ")

In [382]:
df15 = df15.drop(["ДопКод"], axis=1)

In [383]:
df15["категория_пр"] = 3

In [384]:
df15["агрессивность"] += 1
df15["группа_газов"] += 1
df15["материал"] += 1
df15["вп_болты"] += 1

# Part
df15.loc[df15["тип_ЛКМ"] == "ПФ", "тип_ЛКМ"] = 1
df15.loc[df15["тип_ЛКМ"] == "АУ", "тип_ЛКМ"] = 2
df15.loc[df15["тип_ЛКМ"] == "ЭЭ", "тип_ЛКМ"] = 3
df15.loc[df15["тип_ЛКМ"] == "МА", "тип_ЛКМ"] = 4
df15.loc[df15["тип_ЛКМ"] == "ПВ", "тип_ЛКМ"] = 11

In [385]:
df15 = df15.drop_duplicates(subset=["code", "subnode_id"])

In [386]:
df15_operating_cond = df15.copy()

In [387]:
df15 = df15.rename(columns={"код_отд": "department_id",
                            "подп1": "signed1_id",
                            "подп2": "signed2_id",
                            "гл_спец": "chief_specialist_id",
                            "рук_гр": "group_leader_id",
                            "н_контр": "norm_contr_id",
                            "дата_выд": "issue_date",
                            "кол_томов": "num_of_volumes",
                            "дата_ред": "edited_date",
                            "коэф_надежн": "safety_coeff",
                            "t_эксплуат": "operating_temp",
                            "зона_эксплуат": "operating_zone",
                            "группа_газов": "gas_group",
                            "агрессивность": "aggressiveness",
                            "материал": "material",
                            "тип_ЛКМ": "paintwork_type",
                            "прим": "note",
                            "категория_пр": "fire_hazard_category_id",
                            "вп_болты": "high_tensile_bolts",
                            "п_транспорт": "p_transport",
                            "п_площадки": "p_site",
                            "10ХСНД": "xcnd",
                            "ТекстЗдСм": "task_text",
                            "ДопОбъемы": "additional_text",
                            "ВесПоВМП": "vmp_weight",
                            "ВыпЗдСм": "impl_3d_estimate",
                            "m_p": "name"
                       })

In [388]:
df15["issue_date"] = df15["issue_date"].astype(object).where(df15["issue_date"].notnull(), None)
df15["edited_date"] = df15["edited_date"].astype(object).where(df15["edited_date"].notnull(), None)
df15 = df15.where(pd.notnull(df15), None)

In [389]:
df15["chief_specialist_id"] = df15["chief_specialist_id"].replace(0, None)
df15["group_leader_id"] = df15["group_leader_id"].replace(0, None)
df15["norm_contr_id"] = df15["norm_contr_id"].replace(0, None)

In [390]:
df16 = df15.drop(["vmp_weight", "impl_3d_estimate", "xcnd", "task_text", "additional_text", "safety_coeff", "aggressiveness", "operating_temp", "operating_zone", "gas_group", "material", "high_tensile_bolts"], axis=1)

In [391]:
df16 = df16.reset_index(drop=True)

In [392]:
df16["id"].to_csv("mark_ids.csv")

In [393]:
cursor.execute(open("sql_server/7.sql", "r").read())
conn.commit()

In [394]:
execute_values(df16, "marks")

<h1>Условия эксплуатации марки</h1>

In [395]:
df_op_cond = df15_operating_cond.loc[:, ["id", "коэф_надежн", "агрессивность", "t_эксплуат", "зона_эксплуат", "группа_газов", "материал", "тип_ЛКМ", "вп_болты"]]

In [396]:
df_op_cond["t_эксплуат"] = df_op_cond["t_эксплуат"].fillna(-34)

In [397]:
df_op_cond = df_op_cond.rename(columns={"id": "mark_id",
                            "коэф_надежн": "safety_coeff",
                            "t_эксплуат": "temperature",
                            "зона_эксплуат": "operating_area_id",
                            "группа_газов": "gas_group_id",
                            "агрессивность": "env_aggressiveness_id",
                            "материал": "construction_material_id",
                            "тип_ЛКМ": "paintwork_type_id",
                            "вп_болты": "high_tensile_bolts_type_id"
                       })

In [398]:
cursor.execute(open("sql_server/y_mark_operating_conditions.sql", "r").read())
conn.commit()

In [399]:
execute_values(df_op_cond, "mark_operating_conditions", False)

<h1>Задание на смету</h1>

In [400]:
df_task = df15.loc[:, ["id", "task_text", "additional_text"]]

In [401]:
df_task["task_text"] = df_task["task_text"].str.strip()

In [402]:
df['обозн'] = 'Разработать сметную документацию к чертежам ' + df['обозн'].astype(str) + '\r\nСостав и объемы работ:'

In [403]:
df_task.loc[df_task["task_text"].isna(), "task_text"] = list(df.loc[df["код_марки"].isin(df_task.loc[df_task["task_text"].isna(), "id"]), "обозн"])

In [404]:
df_task = df_task.rename(columns={"id": "mark_id"})

In [405]:
df_task["task_text"] = df_task["task_text"].str.replace("\r\n", "\n")

In [406]:
df_task["additional_text"] = df_task["additional_text"].str.replace("\r\n", "\n")

In [407]:
cursor.execute(open("sql_server/y_estimate_task.sql", "r").read())
conn.commit()

In [408]:
execute_values(df_task, "estimate_task", False)

<h1>Выпуски спецификаций</h1>

In [409]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.ВыпСпец', db_conn)

In [410]:
df = df.replace("", np.nan)

In [411]:
df = df.drop_duplicates(subset=["код_марки", "выпуск"])

In [412]:
df = df.drop(["вкл_вып"], axis=1)

In [413]:
df["текущий"] = df["текущий"].map(lambda x: True if (x == 1) else False)

In [414]:
df = df.rename(columns={"Поз_выпуска": "id",
                        "код_марки": "mark_id",
                        "выпуск": "num",
                        "дата_созд": "created_date",
                        "текущий": "is_current",
                        "прим": "note"
                       })

In [415]:
df["created_date"] = df["created_date"].astype(object).where(df["created_date"].notnull(), datetime(2000, 1, 1))
df = df.where(pd.notnull(df), None)

In [416]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()

In [417]:
df = df[df["mark_id"].isin(m_ids)]

In [418]:
df.loc[:, ["id", "mark_id", "num"]].to_csv("specs.csv")

In [419]:
cursor.execute(open("sql_server/3.sql", "r").read())
conn.commit()

In [420]:
execute_values(df, "specifications")

<h1>Листы</h1>

In [421]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Листы', db_conn)

In [422]:
df = df.replace("", np.nan)

In [423]:
df = df[df["номер"].isna() == False]

In [424]:
df["тип_док"] = df["тип_док"] + 1

In [425]:
df["вып"] = df["вып"].map(lambda x: np.nan if x == 0 else x)

In [426]:
df["пров"] = df["пров"].map(lambda x: np.nan if x == 0 else x)

In [427]:
df["норм"] = df["норм"].map(lambda x: np.nan if x == 0 else x)

In [428]:
df = df.drop(["шифр_вып", "шифр_пров", "шифр_норм"], axis=1)

In [429]:
df["формат"] = df["формат"].map(lambda x: 1.0 if np.isnan(x) else x)

In [430]:
df = df.drop_duplicates(subset=["код_марки", "номер", "тип_док"])

In [431]:
df = df[df["вып"].isna() == False]

In [432]:
df["название"] = df["название"].str.strip()

In [433]:
for i in range(0, len(df)):
    try:
        arr = df["название"][i].split()
        new_str = ' '.join(arr)
        df.at[i, "название"] = new_str
    except Exception:
        pass

In [434]:
df = df.rename(columns={"поз_листа": "id",
                        "код_марки": "mark_id",
                        "номер": "num",
                        "название": "name",
                        "формат": "form",
                        "вып": "creator_id",
                        "пров": "inspector_id",
                        "норм": "norm_contr_id",
                        "тип_док": "type_id",
                        "выпуск": "release_num",
                        "листов": "num_of_pages",
                        "прим": "note"
                       })

In [435]:
df = df.where(pd.notnull(df), None)

In [436]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()

In [437]:
e_ids = pd.read_csv("employee_ids.csv")
e_ids = e_ids.drop(["Unnamed: 0"], axis=1)
e_ids = e_ids.values.flatten().tolist()

In [438]:
df = df[df["mark_id"].isin(m_ids)]
df.loc[df["creator_id"].isin(e_ids) == False, "creator_id"] = None
df.loc[df["inspector_id"].isin(e_ids) == False, "inspector_id"] = None
df.loc[df["norm_contr_id"].isin(e_ids) == False, "norm_contr_id"] = None

In [439]:
df["release_num"] = df["release_num"].fillna(0)
df["num_of_pages"] = df["num_of_pages"].fillna(0)

In [440]:
df = df[df["creator_id"].isna() == False]
df = df[df["name"].isna() == False]

In [441]:
cursor.execute(open("sql_server/6.sql", "r").read())
conn.commit()

In [442]:
execute_values(df, "docs")

<h1>Обозначения марок</h1>

In [443]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Об_марок', db_conn)

In [444]:
df = df.iloc[:-1, :]

In [445]:
df.loc[0, "об_марки"] = "КМД"
df.loc[0, "наим_марки"] = "Конструкции металлические деталировочные"

In [446]:
df = df.rename(columns={"Поз": "id",
                        "об_марки": "designation",
                        "наим_марки": "name"
                       })

In [447]:
cursor.execute(open("sql_server/9.sql", "r").read())
conn.commit()

In [448]:
execute_values(df, "mark_designations")

<h1>Прилагаемые документы марки</h1>

In [449]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Од_прил', db_conn)

In [450]:
df = df.replace("", np.nan)

In [451]:
df = df.drop_duplicates(subset=["код_марки", "обозн_прил"])

In [452]:
df = df.where(pd.notnull(df), None)

In [453]:
df = df.rename(columns={"Поз_прил": "id",
                        "код_марки": "mark_id",
                        "наим_прил": "name",
                        "прим": "note",
                        "обозн_прил": "designation"
                       })

In [454]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["mark_id"].isin(m_ids)]

In [455]:
cursor.execute(open("sql_server/12.sql", "r").read())
conn.commit()

In [456]:
execute_values(df, "attached_docs")

<h1>Согласования</h1>

In [457]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Од_согл', db_conn)

In [458]:
df = df.replace("", np.nan)

In [459]:
df = df.drop(["отдел1", "отдел2", "отдел3", "отдел4", "отдел5", "отдел6", "отдел7"], axis=1)

In [460]:
p1 = df.iloc[:, [0,1]]
p1 = p1.rename(columns={"исп1": "исп"})

p2 = df.iloc[:, [0,2]]
p2 = p2.rename(columns={"исп2": "исп"})

p3 = df.iloc[:, [0,3]]
p3 = p3.rename(columns={"исп3": "исп"})

p4 = df.iloc[:, [0,4]]
p4 = p4.rename(columns={"исп4": "исп"})

p5 = df.iloc[:, [0,5]]
p5 = p5.rename(columns={"исп5": "исп"})

p6 = df.iloc[:, [0,6]]
p6 = p6.rename(columns={"исп6": "исп"})

p7 = df.iloc[:, [0,7]]
p7 = p7.rename(columns={"исп7": "исп"})

In [461]:
df = pd.concat([p1, p2, p3, p4, p5, p6, p7], ignore_index=True)

In [462]:
df = df.dropna()
df = df[df["исп"] != 0]

In [463]:
df = df.drop_duplicates()

In [464]:
df.insert(0, 'id', range(1, len(df) + 1))

In [465]:
df = df.rename(columns={"код_марки": "mark_id",
                        "исп": "employee_id"
                       })

In [466]:
m_ids = pd.read_csv("mark_ids.csv")
e_ids = pd.read_csv("employee_ids.csv")

In [467]:
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
e_ids = e_ids.drop(["Unnamed: 0"], axis=1)

In [468]:
m_ids = m_ids.values.flatten().tolist()
e_ids = e_ids.values.flatten().tolist()

In [469]:
df = df[df["mark_id"].isin(m_ids)]
df = df[df["employee_id"].isin(e_ids)]

In [470]:
cursor.execute(open("sql_server/13.sql", "r").read())
conn.commit()

In [471]:
execute_values(df, "mark_approvals")

<h1>Ссылочные документы марки</h1>

In [472]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Од_ссыл', db_conn)

In [473]:
df = df.replace("", np.nan)

In [474]:
df= df[df["шифр_сд"] != "1"]
df["шифр_сд"] = df["шифр_сд"].str.upper()

In [475]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [476]:
for i in range(1, 301):
    df = df.append(pd.Series(), ignore_index=True)

<ipython-input-476-9c8014e24f43>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(), ignore_index=True)


In [477]:
df.loc[:, "код_марки"] = random.choices(m_ids, k=300)
df.loc[:, "шифр_сд"] = random.choices(range(1, 300), k=300)

In [478]:
df = df.drop_duplicates(subset=["код_марки", "шифр_сд"])

In [479]:
df = df.drop(["Поз_сд"], axis=1)

In [480]:
df.insert(0, 'id', range(1, len(df) + 1))

In [481]:
df = df.rename(columns={"код_марки": "mark_id",
                        "шифр_сд": "linked_doc_id",
                        "прим": "note"
                       })

In [482]:
cursor.execute(open("sql_server/14.sql", "r").read())
conn.commit()

In [483]:
execute_values(df, "mark_linked_docs")

<h1>Учет дополнительных работ</h1>

In [484]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Од_вып_доп', db_conn)

In [485]:
df = df.replace("", np.nan)

In [486]:
df = df.drop(["шифр_исп"], axis=1)

In [487]:
df = df[df["код_исп"].isna() == False]

In [488]:
df = df.drop_duplicates(subset=["код_марки", "код_исп"])

In [489]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [490]:
df = df.rename(columns={"Поз_вып_доп": "id",
                        "код_марки": "mark_id",
                        "код_исп": "employee_id",
                        "расчет": "valuation",
                        "заказ": "metal_order",
                       })

In [491]:
cursor.execute(open("sql_server/11.sql", "r").read())
conn.commit()

In [492]:
execute_values(df, "additional_work")

<h1>Типовые конструкции</h1>

In [493]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Тс_типовые', db_conn)

In [494]:
df = df[df["имя_тк"].isna() == False]

In [495]:
df["имя_тк"] = df["имя_тк"].str.strip()

<h2>Типовые наименования типовых конструкций</h2>

In [496]:
st_name_df = pd.DataFrame(data=df["имя_тк"].unique(), columns=["name"])

In [497]:
st_name_df.loc[3, "name"] = "Рельсы крановые КР-70 сталь К63"
st_name_df = st_name_df[st_name_df["name"] != "КР-70"]
st_name_df.loc[5, "name"] = "Рельсы крановые Р-43"
st_name_df = st_name_df[st_name_df["name"] != "Оцинкованная сталь ГОСТ14918-80*"]
st_name_df.loc[11, "name"] = "Рельсы крановые КР-70"
st_name_df.loc[12, "name"] = "Детали крепления крановых рельсов"
st_name_df = st_name_df[st_name_df["name"] != "Крановый рельс КР-70"]
st_name_df.loc[16, "name"] = "Накладка 5 на рельс Р-24 по ГОСТ 8141-56"

In [498]:
st_name_df.insert(0, 'id', range(1, len(st_name_df) + 1))

In [499]:
cursor.execute(open("sql_server/yyy_1.sql", "r").read())
conn.commit()

In [500]:
execute_values(st_name_df, "standard_construction_names")

--------------------

In [501]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [502]:
specs = pd.read_csv("specs.csv")
specs = specs.drop(["Unnamed: 0"], axis=1)

In [503]:
for index, row in df.iterrows():
    try:
        df.loc[index, "specification_id"] = specs.loc[(specs["mark_id"] == row["код_марки"]) & (specs["num"] == row["выпуск"]), "id"].values[0]
    except:
        pass

In [504]:
df = df.drop(["код_марки", "выпуск"], axis=1)

In [505]:
df.loc[24, "кол_тк"] = 1

In [506]:
# https://wikiweight.info/strmat/massa-ankerov-hilti.html
df.loc[28, "вес_тк"] = 0.056
df.loc[32, "вес_тк"] = 0.056

In [507]:
df.loc[25, "вес_тк"] = 0.024
df.loc[29, "вес_тк"] = 0.024
df.loc[25, "кол_тк"] = 360
df.loc[29, "кол_тк"] = 360
df = df[df["сч_тк"] != 54]
df = df[df["сч_тк"] != 58]

In [508]:
df = df.rename(columns={"сч_тк": "id",
                        "имя_тк": "name",
                        "кол_тк": "num",
                        "лист_тк": "sheet",
                        "вес_тк": "weight"
                       })

In [509]:
df = df.where(pd.notnull(df), None)

In [510]:
cursor.execute(open("sql_server/47.sql", "r").read())
conn.commit()

In [511]:
execute_values(df, "standard_constructions")

<h1>Виды конструкций марки</h1>

In [512]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Тс_Виды_к', db_conn)

In [513]:
df = df.drop_duplicates()

In [514]:
df = df.dropna(subset=["кт_окраски", "кол_тип_констр", "имя_к", "Шифр_к", "Шифр_пк"])

In [515]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [516]:
specs = pd.read_csv("specs.csv")
specs = specs.drop(["Unnamed: 0"], axis=1)

In [517]:
df.insert(1, "specification_id", 0, True) 

In [518]:
for index, row in df.iterrows():
    try:
        df.loc[index, "specification_id"] = specs.loc[(specs["mark_id"] == row["код_марки"]) & (specs["num"] == row["выпуск"]), "id"].values[0]
    except:
        pass

In [519]:
df = df[df["specification_id"] != 0]

In [520]:
df = df.drop(["код_марки", "выпуск"], axis=1)

In [521]:
df["Шифр_пк"] = df["Шифр_пк"].replace(0, np.nan)

In [522]:
df = df.drop(["вкл", "окраска", "огнезащита", "ПределОС", "ПредСост_несущ", "ПредСост_целост", "ПредСост_тепло_изол", "ПредСост_тепло_поток", "ПредСост_газ", "статус_к", "ТеорРасходОЗ"], axis=1)

In [523]:
df = df.rename(columns={"сч_констр": "id",
                        "имя_к": "name",
                        "Шифр_к": "type_id",
                        "Шифр_пк": "subtype_id",
                        "расценка": "valuation",
                        "шифр_тип_альбома": "standard_album_code",
                        "кол_тип_констр": "num_of_standard_constructions",
                        "притупление": "has_edge_blunting",
                        "динамика": "has_dynamic_load",
                        "фланцы": "has_flanged_connections",
                        "контроль_сварки": "welding_control_id",
                        "кт_окраски": "paintwork_coeff",
                       })

In [524]:
df = df.where(pd.notnull(df), None)

In [525]:
df = df[df["type_id"] < 88]

In [526]:
df = df[df["type_id"] > 0]

In [527]:
df = df.drop_duplicates(subset=["specification_id", "name", "paintwork_coeff"])

In [528]:
df.loc[:, ["id"]].to_csv("construction_ids.csv")

In [529]:
cursor.execute(open("sql_server/46.sql", "r").read())
conn.commit()

In [530]:
execute_values(df, "constructions")

<h1>Справочник болтов, диаметры</h1>

In [531]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_болтов', db_conn)

In [532]:
df = df.rename(columns={"код_д": "id",
                        "диаметр": "diameter",
                        "масса_г": "nut_weight",
                        "сталь_ш": "washer_steel",
                        "масса_ш": "washer_weight",
                        "толщина_ш": "washer_thickness",
                        "ТУ_б": "bolt_tech_spec",
                        "кл_пр_б": "strength_class",
                        "ТУ_г": "nut_tech_spec",
                        "ТУ_ш": "washer_tech_spec",
                       })

In [533]:
cursor.execute(open("sql_server/20.sql", "r").read())
conn.commit()

In [534]:
execute_values(df, "bolt_diameters")

<h1>Справочник болтов, длины</h1>

In [535]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_болтов_дл', db_conn)

In [536]:
df = df.rename(columns={"Код_болта": "id",
                        "код_д": "diameter_id",
                        "длина_б": "length",
                        "длина_нар": "screw_length",
                        "масса_б": "weight"
                       })

In [537]:
cursor.execute(open("sql_server/21.sql", "r").read())
conn.commit()

In [538]:
execute_values(df, "bolt_lengths")

<h1>Высокопрочные болты видов конструкций</h1>

In [539]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Тс_болты', db_conn)

In [540]:
c_ids = pd.read_csv("construction_ids.csv")
c_ids = c_ids.drop(["Unnamed: 0"], axis=1)
c_ids = c_ids.values.flatten().tolist()

In [541]:
df = df[df["сч_констр"].isin(c_ids)]

In [542]:
df.loc[:, "код_диаметра"] = df.loc[:, "код_диаметра"].replace(0, np.nan)

In [543]:
df = df.dropna()

In [544]:
df = df.rename(columns={"поз_болта": "id",
                        "сч_констр": "construction_id",
                        "код_диаметра": "diameter_id",
                        "пакет": "packet",
                        "кол": "num",
                        "гаек": "nut_num",
                        "шайб": "washer_num",
                       })

In [545]:
cursor.execute(open("sql_server/45.sql", "r").read())
conn.commit()

In [546]:
execute_values(df, "construction_bolts")

<h1>Виды профилей</h1>

In [547]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_видов_п', db_conn)

In [548]:
df["Имя_в"] = df["Имя_в"].str.strip()
df["Прим"] = df["Прим"].str.strip()

In [549]:
df = df.drop_duplicates(subset=["Имя_в"])

In [550]:
df = df.rename(columns={"Шифр_в": "id",
                        "Имя_в": "name",
                        "Прим": "note"
                       })

In [551]:
df = df.sort_values(by=["id"])

In [552]:
df = df.reset_index(drop=True)

In [553]:
df.loc[:, ["id"]].to_csv("prof.csv")

In [554]:
cursor.execute(open("sql_server/23.sql", "r").read())
conn.commit()

In [555]:
execute_values(df, "profile_classes")

<h1>Типы профилей</h1>

In [556]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_типов_проф', db_conn)

In [557]:
df["Имя_т"] = df["Имя_т"].str.strip()

In [558]:
df = df.rename(columns={"Тип": "id",
                        "Имя_т": "name"
                       })

In [559]:
cursor.execute(open("sql_server/42.sql", "r").read())
conn.commit()

In [560]:
execute_values(df, "profile_types")

<h1>Профили</h1>

In [561]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_проф_полн', db_conn)

In [562]:
df = df.replace("", np.nan)

In [563]:
df = df[df["Имя_п"].isna() == False]

In [564]:
df = df[df["Вес"].isna() == False]

In [565]:
df["Имя_п"] = df["Имя_п"].str.strip()
df["Имя_п0"] = df["Имя_п0"].str.strip()

In [566]:
df = df.drop(["Имя_п0", "дл_имя"], axis=1)

In [567]:
df = df[df["Разрешен"]]
df = df.drop(["Разрешен"], axis=1)

In [568]:
df = df[(df["Имя_п"] != "200*160*7") | (df["Символ"].isna() == False)]

In [569]:
df = df.drop_duplicates(subset=["Вид_п", "Имя_п", "Символ"])

In [570]:
df.loc[df["Вид_п"] == 16, "Символ"] = "-"
df.loc[df["Вид_п"] == 17, "Символ"] = "-"

In [571]:
df_2 = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Тс_элементы', db_conn)

In [572]:
df_2["Имя_п"] = df_2["Имя_п"].str.strip()
df_2["Символ"] = df_2["Символ"].str.strip()

In [573]:
c_ids = pd.read_csv("construction_ids.csv")
c_ids = c_ids.drop(["Unnamed: 0"], axis=1)
c_ids = c_ids.values.flatten().tolist()

In [574]:
df_2 = df_2[df_2["Сч_констр"].isin(c_ids)]

In [575]:
p_ids = pd.read_csv("prof.csv")
p_ids = p_ids.drop(["Unnamed: 0"], axis=1)
p_ids = p_ids.values.flatten().tolist()

In [576]:
df_2 = df_2[df_2["Вид_п"].isin(p_ids)]

In [577]:
df_2 = df_2[df_2["Символ"].isna() == False]

In [578]:
df_2 = df_2.drop(["МаркаОЗ", "АрифмВыр"], axis=1)

In [579]:
df_2 = df_2.drop(["ПерекрПерим", "КолВСеч", "Поз", "ГруппаПТ"], axis=1)

In [580]:
for index, row in df_2.iterrows():
    try:
        df_2.loc[index, "profile_id"] = df.loc[(df["Вид_п"] == row["Вид_п"]) & (df["Имя_п"] == row["Имя_п"]) & (df["Символ"] == row["Символ"]), "Профиль"].values[0]
    except Exception as e:
        pass

In [581]:
df_null = df_2.loc[df_2['profile_id'].isna()]

In [582]:
df_null = df_null.drop(["Сч_констр", "Сч_поз", "Сталь", "Длина", "статус", "profile_id"], axis=1)

In [583]:
df_null = df_null.rename(columns={"Вид_п": "class_id",
                        "Имя_п": "name",
                        "Символ": "symbol",
                        "Вес": "weight",
                        "Поверхн": "area",
                        "Тип_п": "type_id"
                       })

In [584]:
df_null.insert(0, 'id', range(1962, len(df_null) + 1962))

In [585]:
df = df.where(pd.notnull(df), None)

In [586]:
df = df.rename(columns={"Профиль": "id",
                        "Вид_п": "class_id",
                        "Имя_п": "name",
                        "Символ": "symbol",
                        "Вес": "weight",
                        "Поверхн": "area",
                        "Тип_п": "type_id"
                       })

In [587]:
df = df.append(df_null, ignore_index = True)

In [588]:
df[df.duplicated(subset=["class_id", "name", "symbol"], keep="first")]
df = df.drop_duplicates(subset=["class_id", "name", "symbol"])

In [589]:
df = df[df["id"] != 3194]

In [590]:
df.loc[df["id"] == 2755, "weight"] = 4.396
df.loc[df["id"] == 2755, "area"] = 0.00288

In [591]:
df.loc[:, ["id", "class_id", "name", "symbol"]].to_csv("profiles.csv")

In [592]:
cursor.execute(open("sql_server/33.sql", "r").read())
conn.commit()

In [593]:
execute_values(df, "profiles")

<h1>Марки стали</h1>

In [594]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_стали', db_conn)

In [595]:
df = df.replace("", np.nan)

In [596]:
df = df.drop([71])

In [597]:
df["Имя_с"] = df["Имя_с"].str.strip()
df["Стандарт"] = df["Стандарт"].str.strip()

In [598]:
df = df.rename(columns={"Шифр_с": "id",
                        "Имя_с": "name",
                        "Стандарт": "standard",
                        "Прочность": "strength"
                       })

In [599]:
df = df.where(pd.notnull(df), None)

In [600]:
cursor.execute(open("sql_server/41.sql", "r").read())
conn.commit()

In [601]:
execute_values(df, "steel")

<h1>Перечень элементов</h1>

In [602]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Тс_элементы', db_conn)

In [603]:
df["Имя_п"] = df["Имя_п"].str.strip()
df["Символ"] = df["Символ"].str.strip()

In [604]:
c_ids = pd.read_csv("construction_ids.csv")
c_ids = c_ids.drop(["Unnamed: 0"], axis=1)
c_ids = c_ids.values.flatten().tolist()

In [605]:
df = df[df["Сч_констр"].isin(c_ids)]

In [606]:
p_ids = pd.read_csv("prof.csv")
p_ids = p_ids.drop(["Unnamed: 0"], axis=1)
p_ids = p_ids.values.flatten().tolist()

In [607]:
df = df[df["Вид_п"].isin(p_ids)]

In [608]:
df.loc[df["Сталь"] == 72, "Сталь"] = 36

In [609]:
df = df[df["Символ"].isna() == False]

In [610]:
df = df.drop(["МаркаОЗ", "АрифмВыр"], axis=1)

In [611]:
df = df.drop(["ПерекрПерим", "КолВСеч", "Поз", "ГруппаПТ"], axis=1)

In [612]:
profiles = pd.read_csv("profiles.csv")
profiles = profiles.drop(["Unnamed: 0"], axis=1)

In [613]:
df.loc[df["Сч_поз"] == 179045, "Имя_п"] = "140*4"
df.loc[df["Сч_поз"] == 179034, "Имя_п"] = "140*4"

In [614]:
for index, row in df.iterrows():
    try:
        df.loc[index, "profile_id"] = profiles.loc[(profiles["class_id"] == row["Вид_п"]) & (profiles["name"] == row["Имя_п"]) & (profiles["symbol"] == row["Символ"]), "id"].values[0]
    except:
        pass

In [615]:
df = df.drop(["Имя_п", "Символ", "Вес", "Поверхн", "Тип_п", "статус", "Вид_п"], axis=1)

In [616]:
df = df.rename(columns={"Сч_констр": "construction_id",
                        "Сч_поз": "id",
                        "Сталь": "steel_id",
                        "Длина": "length"
                       })

In [617]:
cursor.execute(open("sql_server/48.sql", "r").read())
conn.commit()

In [618]:
execute_values(df, "construction_elements")

<h1>Разделы общих указаний пользователя</h1>

In [506]:
df = pd.DataFrame({'name': ["Разработано на основании",
                            "Разработано в соответствии",
                            "Перечень технических регламентов",
                            "Высотная отметка",
                            "Климатические условия",
                            "Конструктивные решения",
                            "Характеристики конструкций",
                            "Материал конструкций",
                            "Указания по разработке КМД и ППР",
                            "Указания по изготовлению и монтажу",
                            "Крепление элементов",
                            "Соединение элементов",
                            "Антикоррозионная защита",
                            "Огнезащита конструкций",
                            "Перечень скрытых видов работ",
                            "Перечень ответственных конструкций",
                            "Коэффициент трудозатрат"]})

In [507]:
df_sections_len = len(df)

In [508]:
df.insert(1, 'order_num', range(1, len(df) + 1))

In [509]:
df2 = df.copy()

In [510]:
df.insert(2, 'user_id', 1)
df2.insert(2, 'user_id', 2)

In [511]:
df = df.append(df2, ignore_index=True)

In [512]:
df.insert(0, 'id', range(1, len(df) + 1))

In [516]:
cursor.execute(open("sql_server/37_2.sql", "r").read())
conn.commit()

In [517]:
execute_values(df, "general_data_sections")

<h1>Пункты и разделы общих указаний марки</h1>

In [51]:
df_sections = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_разделы_н', db_conn)
df_sections_full = df_sections.copy()

In [52]:
df_sections = df_sections.loc[:, ["раздел", "заголовок_кр", "порядок_р"]]

In [53]:
df_sections = df_sections.rename(columns={"раздел": "id",
                        "заголовок_кр": "name",
                        "порядок_р": "order_num",
                       })

In [54]:
df2 = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_пункты_н', db_conn)

In [55]:
df2["текст"] = df2["текст"].str.strip()

In [56]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Оу_пункты_н', db_conn)

In [57]:
df = df[df["включать"] == True]

In [58]:
df["текст_о"] = df["текст_о"].str.strip()

In [59]:
df = df.drop(["включать", "оригинал", "оригинал", "пункт_фикс_о", "нов_пункт", "неразрывать_о"], axis=1)

In [60]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [61]:
df = df[df["пункт_о"].isin(list(df2["пункт"]))]

In [62]:
df3 = pd.DataFrame({'раздел': [], 'текст': [], 'порядок_п': []})
for index, row in df.iterrows():
    df3 = df3.append(df2.loc[df2["пункт"] == df["пункт_о"][index], ["раздел", "текст", "порядок_п"]])

In [63]:
df.reset_index(drop=True, inplace=True)
df3.reset_index(drop=True, inplace=True)
df4 = pd.concat([df, df3], axis=1)

In [64]:
for index, row in df4.iterrows():
    if df4.loc[index, "текст_о"] == None:
        df4.loc[index, "текст_о"] = df4.loc[index, "текст"]

In [65]:
df4 = df4.drop(["пункт_о", "текст"], axis=1)

In [66]:
df4.loc[3, "текст_о"] = '#Конструкции с монтажными сварными соединениями закреплять сначала на временных болтах класса точности С (М16; М20; М24) (выбрать нужное) классов прочности 4.6; 4.8 (выбрать нужное), а затем по проекту.'

In [67]:
df4 = df4.dropna()

In [68]:
df4 = df4.replace("", None)

In [69]:
df4["текст_о"] = df4["текст_о"].map(lambda x: "# " + x[1:] if x[0] == "#" else x)

In [70]:
global_point_num = df4["Поз_пункта"].max() + 1

In [71]:
def changeOrderNumGroupBySection(df, m):
    global global_point_num
    df_list = []
    for s in df["раздел"].unique():
        df2 = df[df["раздел"] == s]
        df2 = df2.sort_values(by=['порядок_п'])
        df2["порядок_п"] = range(2, len(df2) + 2)
        name = df_sections_full.loc[df_sections_full["раздел"] == s, "заголовок"]
        name = list(name)[0]
        df2 = pd.concat([pd.DataFrame([[global_point_num, m, name, s, 1]], columns=['Поз_пункта', 'код_марки', 'текст_о', 'раздел', 'порядок_п']), df2])
        global_point_num += 1
        df_list.append(df2.copy())
    return pd.concat(df_list)

In [72]:
def changeOrderNumGroupByMark(df):
    df_list = []
    section_df_list = []
    id_num = 1
    for m in df["код_марки"].unique():
        df2 = df[df["код_марки"] == m]
        sections = df2['раздел'].unique()
        df3 = df_sections[df_sections["id"].isin(sections)]
        df3 = df3.drop(["id"], axis=1)
        df3.insert(0, 'id', range(id_num, len(df3) + id_num))
        df3.insert(2, 'mark_id', m)
        df3 = df3.drop(["order_num"], axis=1)
        df3.insert(2, 'order_num', range(1, len(df3) + 1))
        section_df_list.append(df3.copy())
        id_num += len(df3)
        df2 = changeOrderNumGroupBySection(df2, m)
        df_list.append(df2.copy())
    return (pd.concat(df_list), pd.concat(section_df_list))

In [73]:
(df5, df6) = changeOrderNumGroupByMark(df4)

In [74]:
cursor.execute(open("sql_server/37_3.sql", "r").read())
conn.commit()

In [75]:
execute_values(df6, "mark_general_data_sections")

In [76]:
df5 = df5.drop(["код_марки"], axis=1)

In [77]:
df5 = df5.reset_index(drop=True)

In [78]:
df5 = df5.rename(columns={"Поз_пункта": "id",
                          "текст_о": "text",
                          "раздел": "section_id",
                          "порядок_п": "order_num",
                       })

In [79]:
cursor.execute(open("sql_server/35_xx.sql", "r").read())
conn.commit()

In [ ]:
execute_values(df5, "mark_general_data_points")

<h1>Пункты общих указаний пользователя</h1>

In [518]:
df = pd.DataFrame({'section_id': [1, 1, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 9, 10, 10, 10, 10, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 15, 15, 15, 15],
                   'text': ["Рабочая документация разработана на основании:",
                            "- поручения от 28.02.2020 № АГЛ-35/0362",
                            "- письма от 05.03.2020 № УПиН-35/0431",
                            "- рекомендаций, выполненного специалистами \"АО МАГНИТОГОРСКИЙ ГИПРОМЕЗ\"",
                            "Рабочая документация разработана в соответствии с техническими условиями, требованиями действующих технических регламентов, стандартов, сводов правил и других документов, содержащих установленные требования.",
                            "Перечень технических регламентов и нормативных документов, в соответствии с требованиями которых разработана рабочая документация",
                            "- Федеральный закон от 30.12.2009 № 384-ФЗ \"Технический регламент о безопасности зданий и сооружений\"",
                            "- Федеральный закон от 22.07.2008 № 123-ФЗ \"Технический регламент о требованиях пожарной безопасности\"",
                            "- СП 16.13330.2017 \"Стальные конструкции. Актуализированная редакция СНиП II-23-81*\"",
                            "- СП 20.13330.2016 \"Нагрузки и воздействия. Актуализированная редакция СНиП 2.01.07-85*\"",
                            "- СП 28.13330.2017 \"Защита строительных конструкций от коррозии. Актуализированная редакция СНиП 2.03.11-85\"",
                            "- СП 53-102-2004 \"Общие правила проектирования стальных конструкций\"",
                            "- СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\"",
                            "- ГОСТ 23118-2012 \"Конструкции стальные строительные. Общие технические условия\"",
                            "- ГОСТ 27751-2014 \"Надежность строительных конструкций и оснований. Основные положения\"",
                            "- ГОСТ 9.402-2004 \"Единая система защиты от коррозии и старения (ЕСЗКС). Покрытия лакокрасочные. Подготовка металлических поверхностей к окрашиванию\"",
                            "Высотные отметки абсолютные.",
                            "Климатические условия",
                            "- климатический район – IB",
                            "- расчетная температура наружного воздуха наиболее холодных суток обеспеченностью 0,98 – минус 43 °C",
                            "- нормативное значение веса снегового покрова на 1 м^2 горизонтальной поверхности земли для г. Магнитогорска (согласно СП 20.13330-2016, приложение K, табл. K1) – 1,3 кПа (130 кг/м^2)",
                            "- нормативное значение ветрового давления для II района (согласно СП 20.13330-2016, п. 11.14, табл. 11.1) – 0,30 кПа (30 кг/м^2)",
                            "- зона влажности – сухая",
                            "- расчетная сейсмическая активность – 5 баллов",
                            "Конструктивные решения",
                            "# Чертежи марки \"КМ\" являются исходным материалом для разработки деталировочных чертежей марки \"КМД\"",
                            "# В данной рабочей документации разработано восстановление пролетных строений (усиление ферм пролетных строений, металлоконструкций по верхним и нижним поясам ферм) и восстановление антикоррозионного покрытия существующих элементов пролетных строений галереи конвейера № 706-1 УУК и КДИ аглоцеха ГОП ПАО \"ММК\"",
                            "# Все работы должны выполняться специализированной организацией, имеющей свидетельство СРО на данный вид деятельности и имеющие допуск по электробезопасности",
                            "# Работы производить по специально разработанному и согласованному в установленном порядке со всеми инспектирующими организациями и эксплуатацией цеха проекту производства работ (ППР) с соблюдением требований действующих нормативных документов и обязательным учетом фактического состояния существующих конструкций",
                            "# При выполнении работ принять меры по обеспечению целостности и сохранности существующих сооружений и коммуникаций, находящихся в зоне производства работ",
                            "# При производстве сварочных работ предусмотреть комплекс организационно-технических мероприятий по обеспечению пожарной безопасности",
                            "# На всех этапах ведения демонтажных и монтажных работ обеспечить устойчивость и неизменяемость положения существующих конструкций",
                            "# Перед началом работ необходимо выполнить очистку конструкций в зоне ремонта от мусора, грязи, технологической пыли",
                            "# Исполнитель несет ответственность за соответствие выполненных работ чертежам. Всякое отступление от чертежей при производстве работ должно быть согласовано с организацией-разработчиком данной рабочей документации",
                            "Характеристики конструкций",
                            "- коэффициент надежности по ответственности – {}",
                            "- степень агрессивного воздействия среды – {}",
                            "- расчетная температура эксплуатации – {}",
                            "Материал конструкций",
                            "Марки сталей элементов конструкций приняты в зависимости от группы конструкций с учетом расчетной температуры и приведены в таблицах \"Ведомость элементов\", на схемах расположения элементов конструкций, на чертежах элементов и узлов конструкций и в спецификации металлопроката",
                            "Указания по разработке КМД и ППР",
                            "Отступления от рабочих чертежей КМ не допускаются. В случае необходимости эти отступления должны быть согласованы с организацией-разработчиком рабочих чертежей марки \"КМ\"",
                            "Указания по изготовлению и монтажу металлоконструкций",
                            "# Изготовление металлоконструкций производить в соответствии с требованиями ГОСТ 23118-2012, СП 53-101-98 \"Изготовление и контроль качества стальных строительных конструкций\", указаниями в чертежах марки \"КМ\"",
                            "# Монтаж металлоконструкций производить в соответствии с требованиями СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\", СНИП 12-04-2002 \"Безопасность труда в строительстве. Часть 2. Строительное производство\" и указаниями в чертежах марки \"КМ\"",
                            "# Отклонения металлоконструкций при изготовлении и монтаже не должны превышать предельных значений, указанных в СП 53-101-98, СП 70.13330.2012 и в чертежах марки \"КМ\" данного основного комплекта",
                            "Крепление элементов",
                            "Минимальное усилие для расчетного прикрепления элементов – 5 тс",
                            "Соединение элементов металлоконструкций",
                            "- сварные",
                            "- на болтах нормальной точности по ГОСТ Р ИСО 4014-2013",
                            "# Материалы для сварки принимать по таблице Г.1 СП 16.13330.2017 в зависимости от марок стали свариваемых элементов",
                            "# Монтажные швы выполнять ручной сваркой ГОСТ 9467-75 \"Электроды покрытые металлические для ручной дуговой сварки конструкционных и теплоустойчивых сталей. Типы\":\nДля сталей C245 сварку производить электродами типа 342A",
                            "# Применение прерывистых швов не допускается",
                            "# Контроль качества монтажных сварных соединений должен осуществляться в соответствии с требованиями п. 10.4 СП 70.13330.2012",
                            "# Болты класса точности B (нормальной точности) M20 приняты по ГОСТ Р ИСО 4014-2013 \"Болты с шестигранной головкой. Классы точности A и B\", класса прочности 8.8 по ГОСТ ISO 898-1-2014 \"Механические свойства крепежных изделий из углеродистых и легированных сталей. Часть 1. Болты, винты и шпильки установленных классов прочности с крупным и мелким шагом резьбы\"",
                            "# Гайки тип 1 класса точности B приняты по ГОСТ ISO 4032-2014. При работе болтов на срез и растяжение класс прочности гаек принят в соответствии с классом прочности болтов: 5 – при 5.6",
                            "Шайбы приняты круглые по ГОСТ 11371-78 исполнения 1 класса точности A",
                            "Для болтов класса точности B диаметр отверстий в конструкциях, работающих на срез и смятие, принят d=dв+2",
                            "В расчетных соединениях для болтов класса точности B разность номинальных диаметров отверстий и болтов не должна превышать 3 мм с учетом предельных отклонений диаметра отверстия и их овальности 0; +0,6; и 0; +1,0 по табл. 2 ГОСТ 23118-2012",
                            "Антикоррозионная защита",
                            "# Антикоррозионная защита запроектирована в соответствии с требованиями СП 28.13330.2017 \"Защита строительных конструкций от коррозии\"",
                            "# Антикоррозионную защиту производить в соответствии с требованиями СП 72.13330.2016 \"Защита строительных конструкций и сооружений от коррозии. СНиП 3.04.03-85\" и ГОСТ 12.3.005-75 \"Система стандартов безопасности труда (ССБТ). Работы окрасочные. Общие требования безопасности\"",
                            "# Подготовка поверхностей перед окраской должна осуществляться в соответствии с требованиями ГОСТ 9.402-2004 путем удаления ржавчины и прокатной окалины с помощью ручного или механизированного абразивного инструмента и очистки жировых и прочих загрязнений с помощью волосяных кистей или ветоши, смоченных в уайт-спирите. Перед нанесением антикоррозионного покрытия поверхности конструкций обеспылить",
                            "# Защита металлоконструкций от коррозии осуществляется окраской лакокрасочными материалами группы III: эмалью XB-7141 по ТУ 2313-016-75351875-2005 в 4 слоя по 2 слоям грунтовки XC-010 по ТУ 6-21-7-89",
                            "Общая толщина покрытия – 160 мкм",
                            "Степень очистки поверхности стальных конструкций от окислов перед окраской – 2 по ГОСТ 9.402-2004",
                            "Допускается замена лакокрасочных материалов на сертифицированные материалы с аналогичными техническими характеристиками",
                            "Цвет окраски антикоррозионного покрытия принимать в соответствии с цветом окраски существующих конструкций",
                            "Перечень видов работ, на которые должны быть составлены акты освидетельствования скрытых работ:",
                            "- подготовка поверхности металлоконструкций под окраску",
                            "- приемка грунтовочного слоя антикоррозионного покрытия",
                            "- визуальный контроль качества сварных швов"],
                   'order_num': [1, 2, 3, 4, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 1, 2, 1, 2, 1, 2, 3, 4, 1, 2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4]})

In [519]:
df2 = df.copy()

In [520]:
df2['section_id'] += df_sections_len

In [521]:
df = df.append(df2, ignore_index=True)

In [522]:
df.insert(0, 'id', range(1, len(df) + 1))

In [523]:
cursor.execute(open("sql_server/35_2.sql", "r").read())
conn.commit()

In [524]:
execute_values(df, "general_data_points")

In [49]:
conn.close()